In [1]:
#imports
import numpy as np
import pandas as pd
import requests
import datetime as dt
import xlwings as xw

In [2]:
#pick up the fixture list and the static data (fixtures uses team ids) from the api 
url = 'https://fantasy.premierleague.com/api/fixtures/'
r = requests.get(url)
fixtures_json = r.json()

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
static_json = r.json()


In [3]:
#from the team static set up a dictionary of team id to team name
teams = static["teams"]
team_codes ={team["id"]:team["name"] for team in teams}


In [4]:
#Loop through the fixtures and put in a dataframe. Can't parse the date as it has nulls !
fixture_list = []
for i,fixture_details in enumerate((fixture['team_h'],fixture['team_a'],fixture['kickoff_time']) for fixture in fixtures_json):
    fixture_list.append([team_codes[fixture_details[0]],team_codes[fixture_details[1]],fixture_details[2]])

#tidy up the dataframe and parse the dates (works beter outside the loop)    
dfFixtures = pd.DataFrame(fixture_list,columns=["Home","Away","Date/Time"])
dfFixtures["Date/Time"] = pd.to_datetime(dfFixtures["Date/Time"],format="%Y-%m-%dT%H:%M:%SZ")

In [5]:
team = "Spurs"
teamFixtures = dfFixtures[(dfFixtures["Home"]==team) | (dfFixtures["Away"]==team)]

display(teamFixtures.style.hide_index())
sht = xw.Book().sheets[0]
sht.range('A1').options(index=False).value = teamFixtures


Home,Away,Date/Time
Spurs,Everton,2020-09-12 14:00:00
Southampton,Spurs,2020-09-19 14:00:00
Spurs,Newcastle,2020-09-26 14:00:00
Man Utd,Spurs,2020-10-03 14:00:00
Spurs,West Ham,2020-10-17 14:00:00
Burnley,Spurs,2020-10-24 14:00:00
Spurs,Brighton,2020-10-31 15:00:00
West Brom,Spurs,2020-11-07 15:00:00
Spurs,Man City,2020-11-21 15:00:00
Chelsea,Spurs,2020-11-28 15:00:00
